## Task 2. Neural Network Models for Combined Classification and Regression
In this task, we aim to develop and evaluate neural network models capable of performing both classification and regression tasks simultaneously. 

In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
def enableGPU():
    print(tf.version.GIT_VERSION, tf.version.VERSION)
    print(tf.test.is_built_with_cuda())
    #tf.debugging.set_log_device_placement(True)
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      try:
        for gpu in gpus:
          # tf.config.experimental.set_memory_growth(gpu, True)
          tf.config.set_logical_device_configuration(gpus[0],[tf.config.LogicalDeviceConfiguration(memory_limit=5024)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        print(e)
enableGPU()

v2.10.0-76-gfdfc646704c 2.10.1
True
1 Physical GPUs, 1 Logical GPUs


In [3]:
import os
import cv2
import random
import numpy as np 
import pandas as pd
import subprocess as sp
import matplotlib.pyplot as plt
from tensorflow.keras import  layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Model
from keras.layers import Input, Dense, concatenate, Flatten, Conv2D, MaxPooling2D
import gc

In [4]:
class MyCustomCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()

In [5]:
train_path = "C:\\Users\\Tomi\\Desktop\\archive\\fruits-360_dataset\\fruits-360\\Training"
test_path = "C:\\Users\\Tomi\\Desktop\\archive\\fruits-360_dataset\\fruits-360\\Test"

In [6]:
def create_model():
    model = models.Sequential()
    model.add(Conv2D(filters=64, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu', input_shape=(100,100,3)))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=192, kernel_size=(5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=384, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(layers.Dense(1)) 
    return model

In [8]:
# Define image size and batch size
IMAGE_SIZE = (100, 100)  # Fruits 360 images are 100x100
BATCH_SIZE = 32

# Define paths to your dataset
TRAIN_DATA_PATH = train_path
VALIDATION_DATA_PATH = test_path

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
 
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DATA_PATH,
    # shuffle=True,
    subset="validation",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
test_generator = validation_datagen.flow_from_directory(
    VALIDATION_DATA_PATH,
    # shuffle=True,
    subset="training",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

NumClasses = train_generator.num_classes

Found 67692 images belonging to 131 classes.
Found 11335 images belonging to 131 classes.
Found 11353 images belonging to 131 classes.


In [41]:
checkpoint_callback = ModelCheckpoint(filepath='model_weights_epoch_REG{epoch:02d}.h5', save_weights_only=True, save_freq='epoch')
model = create_model()
model.summary()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mae', metrics=['mae'])
model.fit(train_generator, validation_data=validation_generator, epochs=10, callbacks=[checkpoint_callback], verbose=1)
model.save('REG_cnn.h5')

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_58 (Conv2D)          (None, 23, 23, 64)        23296     
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 11, 11, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_59 (Conv2D)          (None, 11, 11, 192)       307392    
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 5, 5, 192)        0         
 g2D)                                                            
                                                                 
 conv2d_60 (Conv2D)          (None, 5, 5, 384)         663936    
                                                                 
 conv2d_61 (Conv2D)          (None, 5, 5, 256)       

In [44]:
model.load_weights('model_weights_epoch_REG10.h5')
scores = model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

355/355 [==============================] - 14s 40ms/step - loss: 0.0077 - mae: 0.0077
evaluate mae: 0.77%
